# Answer Verifier Discriminator(AVD)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib as plt
import seaborn as sns
import re
import string
import operator
from collections import defaultdict
import ast
import pickle

import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

###Loading Data

In [ ]:
# downloading RACE Dataset
!wget http://www.cs.cmu.edu/~glai1/data/race/RACE.tar.gz

--2023-11-10 17:29:16--  http://www.cs.cmu.edu/~glai1/data/race/RACE.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25443609 (24M) [application/x-gzip]
Saving to: ‘RACE.tar.gz’

RACE.tar.gz         100%[===================>]  24.26M  3.18MB/s    in 8.2s    

2023-11-10 17:29:25 (2.96 MB/s) - ‘RACE.tar.gz’ saved [25443609/25443609]



In [ ]:
# extracting dataset
import tarfile
with tarfile.open('/content/RACE.tar.gz') as tar:

    tar.extractall()

In [ ]:
import os

def load_race(location):

    file_ = location
    race = pd.DataFrame()

    for txt in os.listdir(file_):
        with open(location + txt, 'r') as f:
            race= pd.concat([race, pd.read_json(f.read())], ignore_index=True)

    return race

In [ ]:
race_train_high= load_race('/content/RACE/train/high/')
race_dev_high= load_race('/content/RACE/dev/high/')
race_test_high= load_race('/content/RACE/test/high/')

race_train_medium= load_race('/content/RACE/train/middle/')
race_dev_medium= load_race('/content/RACE/dev/middle/')
race_test_medium= load_race('/content/RACE/test/middle/')

In [ ]:
def data_cleaning(context):
    stop_words = set(stopwords.words('english'))
    corpus= []

        # for every line the corpus
    for i in context:
        # it will clean the text line by line
        line = clean_text(i)
        # if the text is empty it will continue the loop
        if line == ' ': continue
        #tokenizing
        line= word_tokenize(line)
        # removing stopwords
        line = [word for word in line if word not in stop_words]
        line = ' '.join(line)
        corpus.append(line)

    return corpus

In [ ]:
# cleaning and preprocessing context data

race_train_high['cleaned_article'] = data_cleaning(race_train_high['article'])
race_dev_high['cleaned_article'] = data_cleaning(race_dev_high['article'])
race_test_high['cleaned_article'] = data_cleaning(race_test_high['article'])

race_train_medium['cleaned_article'] = data_cleaning(race_train_medium['article'])
race_dev_medium['cleaned_article'] = data_cleaning(race_dev_medium['article'])
race_test_medium['cleaned_article'] = data_cleaning(race_test_medium['article'])

In [ ]:
pickle.dump((race_train_high, race_dev_high, race_test_high, race_train_medium, race_dev_medium, race_test_medium),
            open('/content/drive/MyDrive/my assignments/33. A12 Reasoning Challenge- Self case study 2/race_datasets.pkl','wb'))

In [ ]:
race_train_high, race_dev_high, race_test_high, race_train_medium, race_dev_medium, race_test_medium= \
pickle.load(open('/content/drive/MyDrive/my assignments/33. A12 Reasoning Challenge- Self case study 2/race_datasets.pkl', 'rb'))

In [ ]:
race_train_high.head(3)

,answers,options,questions,article,id,cleaned_article
0,B,"[there are few doctors there, the nearest doctor is sometimes very far away from them, there is always heavy traffic on the road, they don't want to see a doctor]",Some people in the Australian outback can't get to a doctor quickly. Because _,"Most people go to a doctor in their own town or suburb . But people in the Australian _ can't get to a doctor quickly. The nearest doctor is sometimes hundreds of kilometers away so they have to call him on a two-way radio. This special doctor is called the ""flying doctor"". He visits sick people by plane.\nWhen someone is very sick, the doctor has to fly to the person's home. His plane lands on a flat piece of ground near the person's house. Sometimes the doctor has to take the patient to hospital. Flying doctors take about 8,600 people to hospital each year.\nHowever, most of the time the person isn't very sick, and the doctor doesn't have to visit. He can give advice on the radio from the office at the flying doctor center. He can tell the patient to use some medicine from a special medicine chest . There is one of these chests in every home in the outback. Each bottle, tube and packet in the chest has a number. The doctor often says something like this,"" take two tablets from bottle 5 every four hours.""",high9717.txt,people doctor town suburb people australian get doctor quickly nearest doctor sometimes hundreds kilometers away call two way radio special doctor called flying doctor visit sick people plane someone sick doctor fly person home plane lands flat piece ground near person house sometimes doctor take patient hospital flying doctors take people hospital year however time person sick doctor visit give advice radio office flying doctor center tell patient use medicine special medicine chest one chests every home outback bottle tube packet chest number doctor often says something like take two tablets bottle every four hours
1,B,"[by sea, by air, on foot, in a car]",The doctor there usually goes to visit his patient _ .,"Most people go to a doctor in their own town or suburb . But people in the Australian _ can't get to a doctor quickly. The nearest doctor is sometimes hundreds of kilometers away so they have to call him on a two-way radio. This special doctor is called the ""flying doctor"". He visits sick people by plane.\nWhen someone is very sick, the doctor has to fly to the person's home. His plane lands on a flat piece of ground near the person's house. Sometimes the doctor has to take the patient to hospital. Flying doctors take about 8,600 people to hospital each year.\nHowever, most of the time the person isn't very sick, and the doctor doesn't have to visit. He can give advice on the radio from the office at the flying doctor center. He can tell the patient to use some medicine from a special medicine chest . There is one of these chests in every home in the outback. Each bottle, tube and packet in the chest has a number. The doctor often says something like this,"" take two tablets from bottle 5 every four hours.""",high9717.txt,people doctor town suburb people australian get doctor quickly nearest doctor sometimes hundreds kilometers away call two way radio special doctor called flying doctor visit sick people plane someone sick doctor fly person home plane lands flat piece ground near person house sometimes doctor take patient hospital flying doctors take people hospital year however time person sick doctor visit give advice radio office flying doctor center tell patient use medicine special medicine chest one chests every home outback bottle tube packet chest number doctor often says something like take two tablets bottle every four hours
2,C,"[tells him to have a rest, tells him what food to eat, gives him some advice, gives him some medicine]","If the person isn't very sick, the doctor only _ .","Most people go to a doctor in their own town or suburb . But people in the Australian _ can't get to a doc

### Labels Transformation



In [ ]:
def get_labels(dataset):
# Initialize empty lists for DataFrame columns
    questions = []
    choices = []
    contexts = []
    labels = []

    ans={'A':1,
         'B':2,
         'C':3,
         'D':4}

    # Process each example in the RACE dataset
    for index, row in dataset.iterrows():
        # Create rows for each choice
        questions.append(row["questions"])
        choices.append(row["options"])
        contexts.append(row["cleaned_article"])
        labels.append(ans[row['answers']])

    # Create a DataFrame from the lists
    df = pd.DataFrame({
        "Question": questions,
        "Choice": choices,
        "Context": contexts,
        "Label": labels
    })

    return df

In [ ]:
# it will transform alphabetical labels to numerical formal
new_race_train_high=  get_labels(race_train_high)
new_race_dev_high= get_labels(race_dev_high)
new_race_test_high= get_labels(race_test_high)

new_race_train_medium= get_labels(race_train_medium)
new_race_dev_medium= get_labels(race_dev_medium)
new_race_test_medium= get_labels(race_test_medium)

## Getting Model Ready

In [ ]:
# here we are downloading preprocessed saved RACE dataset from Gdrive
!gdown 1SrFfAO_RfjC7ckNhJDCJzhWwNekBOd0i

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
# import matplotlib as plt
# import seaborn as sns
# import re
# import string
# import operator
# from collections import defaultdict
# import ast
import pickle

# # import nltk
# from nltk import word_tokenize, pos_tag, ne_chunk
# # from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords

# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')

In [ ]:
# pickle load data
race_train_high, race_dev_high, race_test_high, race_train_medium, race_dev_medium, race_test_medium= \
pickle.load(open('/content/race_datasets.pkl', 'rb'))

In [ ]:
! pip install transformers evaluate datasets sentencepiece
! pip install cloud-tpu-client==0.10
# ! pip install https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
! pip install accelerate==0.26.1
! pip install torch~=2.3.0 torch_xla[tpu]~=2.3.0 -f https://storage.googleapis.com/libtpu-releases/index.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Looking in links: https://stora

In [ ]:
import tensorflow as tf
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import os

In [ ]:
model_name = 'google/bigbird-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Calculate percentiles
percentiles = np.percentile([len((i).split()) for i in race_train_high['cleaned_article']] , range(90, 101))

# Print the percentile
for p, percentile_value in zip(range(90, 101), percentiles):
    print(f"{p}th percentile: {percentile_value}")

90th percentile: 196.0
91th percentile: 199.0
92th percentile: 202.0
93th percentile: 207.0
94th percentile: 212.0
95th percentile: 218.0
96th percentile: 228.0
97th percentile: 240.0
98th percentile: 263.0
99th percentile: 303.0
100th percentile: 566.0


In [ ]:
# Calculate percentiles
percentiles = np.percentile([len((i).split()) for i in race_train_high['questions']] , range(90, 101))

# Print the percentile
for p, percentile_value in zip(range(90, 101), percentiles):
    print(f"{p}th percentile: {percentile_value}")

90th percentile: 15.0
91th percentile: 15.0
92th percentile: 15.0
93th percentile: 16.0
94th percentile: 16.0
95th percentile: 16.0
96th percentile: 17.0
97th percentile: 18.0
98th percentile: 19.0
99th percentile: 20.0
100th percentile: 63.0


In [ ]:
# Calculate percentiles
percentiles = np.percentile([len((i[0]).split()) for i in race_train_high['options']] , range(90, 101))

# Print the percentile
for p, percentile_value in zip(range(90, 101), percentiles):
    print(f"{p}th percentile: {percentile_value}")

90th percentile: 10.0
91th percentile: 10.0
92th percentile: 11.0
93th percentile: 11.0
94th percentile: 11.0
95th percentile: 12.0
96th percentile: 12.0
97th percentile: 13.0
98th percentile: 13.0
99th percentile: 15.0
100th percentile: 91.0


we did percentile check to find out length of the most the text. Length of most of the context lies below 300 and length of most of the question and option lies in between 15-20.
here we are taking 350 max length of the text by combining all three of them.

In [ ]:
from tqdm import tqdm

def tokenizing_df(dataset):

    max_seq_length = 350
    input_ids= np.zeros((len(dataset), 4, max_seq_length), dtype="int32")
    token_type_ids= np.zeros((len(dataset), 4, max_seq_length), dtype="int32")
    attention_mask= np.zeros((len(dataset), 4, max_seq_length), dtype="int32")
    labels= []

    ans={'A': 0,
        'B': 1,
        'C': 2,
        'D': 3}
    # input_ids, token_type_ids, attention_mask, labels= [], [], [], []
    for i, row in tqdm(dataset.iterrows()):

        # adding [CLS] at the begining and [SEP] at the end of the sentence
        # input_ids_1, token_type_ids_1, attention_mask_1= [], [], []

        for j, option in enumerate(row['options']):

            pair = row['questions'] + '[SEP]' +  option + '[SEP]' + row["cleaned_article"]

            tokens = tokenizer.encode(pair)

            input_ids[i, j, :]= np.array(tokens + [0]*(max_seq_length-len(tokens)))[:350]

            # token_type_ids[i, j, :]= np.array([0]* max_seq_length)[:350]

            attention_mask[i, j, :]= np.array([1]* len(tokens) + [0]* (max_seq_length-len(tokens)))[:350]

        # input_ids.append(input_ids_1)
        # token_type_ids.append(token_type_ids_1)
        # attention_mask.append(attention_mask_1)


        labels.append(ans[row['answers']])


    df ={
        "input_ids": input_ids,
        # "token_type_ids": token_type_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


    return df


In [ ]:
# function to tokenize data

def tokenizing_df(data):

    # we are taking 350 sequence 95 percentile of context length
    max_seq_length = 350

    # labels = np.zeros((len(data), 4), dtype="int32")

    ans={'A': 0,
         'B': 1,
         'C': 2,
         'D': 3}

    # adding [CLS] at the begining and [SEP] at the end of the sentence
    #pair =  [data['questions'] + [' <sep> '] + data["cleaned_article"]] * 4
    first_sentences = [[data['questions'][i] + '<s>' + data["cleaned_article"][i]] * 4 for i in range(len(data['questions']))]

    first_sentences = sum(first_sentences, [])
    options= sum(data['options'], [])


    # converting tokens to unique IDs
    tokens = tokenizer(first_sentences, options, max_length= max_seq_length, padding= 'max_length',
                        truncation= True, return_tensors="np")

    # return tokens, np.array(labels)
    df= {
        k: [v[i : i + 4] for i in range(0, len(v), 4)]
        for k, v in tokens.items()
    }

    # labels = []
    df['labels'] =  tf.convert_to_tensor([ans[i] for i in data['answers']], dtype=tf.int32)
    # df['labels'] =  tf.convert_to_tensor([lst[ans[i]] for idx, labels in data['answers']], dtype=tf.int32)
    # for idx, label in enumerate(data['answers']):
    #   lst = np.zeros(4, dtype ='int32')
    #   lst[ans[label]] = 1
    #   labels.append(lst)
    # df['labels'] = labels

    return df

In [ ]:
# concatinating high and medium dataset from all category

race_train = Dataset.from_pandas(pd.concat([race_train_high, race_train_medium], axis=0).reset_index(drop=True))
race_dev = Dataset.from_pandas(pd.concat([race_dev_high, race_dev_medium], axis=0).reset_index(drop=True))
race_test = Dataset.from_pandas(pd.concat([race_test_high, race_test_medium], axis=0).reset_index(drop=True))

In [ ]:
# tokenizing
race_train =  race_train.map(tokenizing_df, batched =True,  batch_size= 10, remove_columns= ['answers',	'options',	'questions',	'article',	'id',	'cleaned_article'])
race_dev = race_dev.map(tokenizing_df, batched =True, batch_size= 10,  remove_columns= ['answers',	'options',	'questions',	'article',	'id',	'cleaned_article'])
race_test = race_test.map(tokenizing_df, batched =True,  batch_size= 10, remove_columns= ['answers',	'options',	'questions',	'article',	'id',	'cleaned_article'])

Map:   0%|          | 0/87866 [00:00<?, ? examples/s]

Map:   0%|          | 0/4887 [00:00<?, ? examples/s]

Map:   0%|          | 0/4934 [00:00<?, ? examples/s]

#### loading data

In [ ]:
pickle.dump((race_train, race_dev, race_test),
            open('/content/drive/MyDrive/race_tokenized_bigbird_350.pkl','wb'))

In [ ]:
race_train, race_dev, race_test= \
pickle.load(open('/content/drive/MyDrive/race_tokenized_bigbird_350.pkl', 'rb'))

## Model Training

In [ ]:
# os.environ['XLA_USE_BF32'] = "1"
# os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

import torch
# import torch_xla
from torch.utils.data import DataLoader

from  accelerate import  Accelerator, DistributedType
from accelerate import notebook_launcher
!pip install torchmetrics
import torchmetrics
from sklearn.metrics import accuracy_score
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
    set_seed,
)

from tqdm.auto import tqdm
import torch_xla.utils.serialization as xser
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import datasets
import transformers
# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.debug.metrics as met
# import torch_xla.utils.utils as xu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 5.4 MB/s eta 0:00:00


In [ ]:
# converting data to pytorch tensor
race_train.set_format("torch")
race_dev.set_format("torch")
race_test.set_format("torch")

In [ ]:
from transformers import BigBirdConfig, AutoModelForMultipleChoice

config = BigBirdConfig.from_pretrained(model_name)
config.attention_type = "original_full"

model = AutoModelForMultipleChoice.from_pretrained(model_name, config=config)


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of BigBirdForMultipleChoice were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# setting hyperparameters as mentioned by the author
hyperparameters = {
    "learning_rate": 2e-5,
    "num_epochs": 1, # due to time restriction for the TPU usage in google colab we are training for 1 epoch
    "train_batch_size": 32, # taking 1 batch size due to fit in a memory
    "eval_batch_size": 1,
    "max_grad": 1.0,
    "adam_epsilon": 1e-6,
    "num_warmup_steps": 1000,
    "seed": 42,
}

In [ ]:
# creating dataloader to prepare data set for model training
def create_dataloaders(train_batch_size, eval_batch_size):
    train_dataloader = DataLoader(
        race_train, shuffle=True, batch_size=train_batch_size
    )
    eval_dataloader = DataLoader(
        race_dev, shuffle=False, batch_size=eval_batch_size
    )
    return train_dataloader, eval_dataloader

In [ ]:
train_dataloader, eval_dataloader = create_dataloaders(
    train_batch_size=hyperparameters["train_batch_size"], eval_batch_size=hyperparameters["eval_batch_size"])

#### experiment with large model

In [ ]:
import gc
gc.collect()

torch.cuda.empty_cache()

In [ ]:
def training_function(model):
    # Initialize accelerator
    accelerator = Accelerator()

    # To have only one message (and not 8) per logs of Transformers or Datasets, we set the logging verbosity
    # to INFO for the main process only.
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    train_dataloader, eval_dataloader = create_dataloaders(
        train_batch_size=hyperparameters["train_batch_size"], eval_batch_size=hyperparameters["eval_batch_size"]
    )
    # The seed need to be set before we instantiate the model, as it will determine the random head.
    set_seed(hyperparameters["seed"])

    # Instantiate optimizer with learning rate and adam epsilon
    optimizer = AdamW(params=model.parameters(), lr=hyperparameters["learning_rate"],
                      eps= hyperparameters['adam_epsilon'], no_deprecation_warning=True)

    # Prepare everything
    # There is no specific order to remember, we just need to unpack the objects in the same order we gave them to the
    # prepare method.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    num_epochs = hyperparameters["num_epochs"]
    # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=hyperparameters['num_warmup_steps'],
        num_training_steps=len(train_dataloader) * num_epochs,
    )
    # Instantiate a progress bar to keep track of training. Note that we only enable it on the main
    # process to avoid having 8 progress bars.
    accelerator.print('Training started...')
    progress_bar = tqdm(range(num_epochs * len(train_dataloader)), disable=not accelerator.is_main_process)

    # Now we train the model
    for epoch in range(num_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            torch.nn.utils.clip_grad_norm_(model.parameters(), hyperparameters["max_grad"])
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

            if step % 100 == 0 or step == len(train_dataloader)-1:
              accelerator.print(f'Epoch: {epoch+1}, Step: {step}, Loss: {loss.item()}')

            # if step == 30:
            #   break
        # initializing model for evaluation
        model.eval()
        all_predictions = []
        all_labels = []

        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                outputs = model(**batch)

            predictions = outputs.logits.argmax(dim=-1)

            # We gather predictions and labels from the 8 TPUs to have them all.
            all_predictions.append(accelerator.gather(predictions))
            all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.
        all_predictions = torch.cat(all_predictions)[:len(race_dev)]
        all_labels = torch.cat(all_labels)[:len(race_dev)]

        # first we have to get all the metrices to cpu and then we will convert to numpy array
        # now we will accuracy score out of both
        eval_metric = accuracy_score(all_labels.cpu().numpy(), all_predictions.cpu().numpy())

        # # Use accelerator.print to print only on the main process.
        accelerator.print(f"Epoch: {epoch+1}, Accuracy_score= {eval_metric}, Loss= {loss.item()}")

    # at the end we will unwrap model from all the TPU cores
    # and then will save model's trained weights
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)

    xser.save(unwrapped_model.state_dict(), f"/content/drive/MyDrive/model.bin", master_only=True)
    # accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/ARD on Race pretrained model/model.pt')
    # torch.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/ARD on Race pretrained model/')
    # unwrapped_model.save_pretrained('/content/drive/MyDrive/ARD on Race pretrained model/', save_function=accelerator.save, state_dict=accelerator.get_state_dict(model))


In [ ]:
%%time

notebook_launcher(training_function, (model,))

In [ ]:
print("Available distributed types:", DistributedType.XLA)

Available distributed types: DistributedType.XLA


In [ ]:
model.load_state_dict(xser.load(f"model.bin"))

####poo

In [ ]:
import gc
gc.collect()

torch.cuda.empty_cache()

In [ ]:
def training_function(model):
    # Initialize accelerator
    accelerator = Accelerator()

    # To have only one message (and not 8) per logs of Transformers or Datasets, we set the logging verbosity
    # to INFO for the main process only.
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    train_dataloader, eval_dataloader = create_dataloaders(
        train_batch_size=hyperparameters["train_batch_size"], eval_batch_size=hyperparameters["eval_batch_size"]
    )
    # The seed need to be set before we instantiate the model, as it will determine the random head.
    set_seed(hyperparameters["seed"])

    # Instantiate optimizer with learning rate and adam epsilon
    optimizer = AdamW(params=model.parameters(), lr=hyperparameters["learning_rate"],
                      eps= hyperparameters['adam_epsilon'], no_deprecation_warning=True)

    # Prepare everything
    # There is no specific order to remember, we just need to unpack the objects in the same order we gave them to the
    # prepare method.
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader
    )

    num_epochs = hyperparameters["num_epochs"]
    # Instantiate learning rate scheduler after preparing the training dataloader as the prepare method
    # may change its length.
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=hyperparameters['num_warmup_steps'],
        num_training_steps=len(train_dataloader) * num_epochs,
    )
    # Instantiate a progress bar to keep track of training. Note that we only enable it on the main
    # process to avoid having 8 progress bars.
    accelerator.print('Training started...')
    progress_bar = tqdm(range(num_epochs * len(train_dataloader)), disable=not accelerator.is_main_process)

    # Now we train the model
    for epoch in range(num_epochs):
        model.train()
        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            torch.nn.utils.clip_grad_norm_(model.parameters(), hyperparameters["max_grad"])
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

            # print loss after each 1000 steps
            if step % 1000 == 0 or step == len(train_dataloader)-1:
              accelerator.print(f'Epoch: {epoch+1}, Step: {step}, Loss: {loss.item()}')


        # initializing model for evaluation
        model.eval()
        all_predictions = []
        all_labels = []

        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                outputs = model(**batch)

            predictions = outputs.logits.argmax(dim=-1)

            # We gather predictions and labels from the 8 TPUs to have them all.
            all_predictions.append(accelerator.gather(predictions))
            all_labels.append(accelerator.gather(batch["labels"]))

        # Concatenate all predictions and labels.
        # The last thing we need to do is to truncate the predictions and labels we concatenated
        # together as the prepared evaluation dataloader has a little bit more elements to make
        # batches of the same size on each process.
        all_predictions = torch.cat(all_predictions)[:len(race_dev)]
        all_labels = torch.cat(all_labels)[:len(race_dev)]

        # first we have to get all the metrices to cpu and then we will convert to numpy array
        # now we will accuracy score out of both
        eval_metric = accuracy_score(all_labels.cpu().numpy(), all_predictions.cpu().numpy())

        # # Use accelerator.print to print only on the main process.
        accelerator.print(f"Epoch: {epoch+1}, Accuracy_score= {eval_metric}, Loss= {loss.item()}")

    # at the end we will unwrap model from all the TPU cores
    # and then will save model's trained weights
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/ARD on Race pretrained model/model.pt')
    # torch.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/ARD on Race pretrained model/')
    # unwrapped_model.save_pretrained('/content/drive/MyDrive/ARD on Race pretrained model/', save_function=accelerator.save, state_dict=accelerator.get_state_dict(model))


In [ ]:
%%time

notebook_launcher(training_function, (model,))

Launching a training on 8 TPU cores.
Training started...


  0%|          | 0/10984 [00:00<?, ?it/s]

Epoch: 1, Step: 0, Loss: 1.4211560487747192
Epoch: 1, Step: 1000, Loss: 1.5400097370147705
Epoch: 1, Step: 2000, Loss: 1.1186223030090332
Epoch: 1, Step: 3000, Loss: 1.1558825969696045
Epoch: 1, Step: 4000, Loss: 0.5302114486694336
Epoch: 1, Step: 5000, Loss: 7.061692714691162
Epoch: 1, Step: 6000, Loss: 0.3434958755970001
Epoch: 1, Step: 7000, Loss: 0.0021830059122294188
Epoch: 1, Step: 8000, Loss: 0.002069066045805812
Epoch: 1, Step: 9000, Loss: 1.275923490524292
Epoch: 1, Step: 10000, Loss: 1.740681767463684
Epoch: 1, Step: 10983, Loss: 0.26758986711502075
Epoch: 1, Accuracy_score= 0.5281358706773072, Loss= 0.27765798568725586
CPU times: user 1min 57s, sys: 15.8 s, total: 2min 13s
Wall time: 5h 20min 18s


## inference

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/AVD on Race pretrained model/model.pt'))

<All keys matched successfully>

In [ ]:
# downloading Arc Dataset with BM25 and DRD scores
!gdown 1yjnzeFxx83Z0AWGjk8B4rPp2Sl12bUFX

Downloading...
From: https://drive.google.com/uc?id=1yjnzeFxx83Z0AWGjk8B4rPp2Sl12bUFX
To: /content/arc_datasets_scores_easy.pkl
100% 96.9M/96.9M [00:00<00:00, 117MB/s] 


In [ ]:
# loading the dataset
train_challenge, dev_challenge, test_challenge=
pickle.load(open('/content/arc_datasets_scores_challenge.pkl', 'rb'))

train_easy, dev_easy, test_easy= \
pickle.load(open('/content/arc_datasets_scores_easy.pkl', 'rb'))

In [ ]:
def joining_scores(data, new_scores):
  '''
  function to join existing DRD scores with AVD scores
  '''
    old_scores = data['score']
    scores =[]
    for idx_1, score_1 in enumerate(old_scores):
        list_of_scores= []
        for idx_2, score_2 in enumerate(score_1):

            list_of_scores.append([score_3 + [new_scores[idx_1][idx_2][idx_3]] for idx_3, score_3 in enumerate(score_2)])
        scores.append(list_of_scores)

    return scores

### Generating AVD scores

In [ ]:
from tqdm import tqdm

def avd_score(model, tokenizer, data):
  '''
  function to get AVD scores from downstream model trained on RACE data set
  '''
    # device = xm.xla_device()
    device = torch.device('cuda')
    model = model.to(device)
    model.eval()


    question= data['only_questions']
    answer= data['only_answers']
    context = data['context']
    scores = []

    # it will iterate through each question
    for i, que in enumerate(tqdm(question)):
        list_of_scores= []
        # it will iterate through each answer of that particular question
        # each question has four option so it will iterate four times
        for j, ans in enumerate(answer[i]):
            # it will get contexts of particular question with particular option
            # it will get 50 contexts
            context_len = len(context[i][j])

            # we will pair question and answer and multiply with 50 times to match context length
            pair= [que + '</s>' + ans] * context_len

            # now we have 50 pairs of ques and ans and 50 contexts
            # now we will tokenize them
            tokens= tokenizer(pair, context[i][j], padding=True, truncation=True, return_tensors="pt").to(device)

            input_ids= torch.unsqueeze(tokens['input_ids'], dim=0).to(device)
            attention_mask= torch.unsqueeze(tokens['attention_mask'], dim=0).to(device)

            # this below code will get softmax scores of each question, answer and context pair which is 50
            with torch.no_grad():

              logits= model(input_ids, attention_mask).logits
            logits.detach()
            softmax = torch.nn.functional.softmax(logits[0], dim= -1)
            list_of_scores.append(softmax.tolist())

        # now we will append softmax scores to the list of all 50 scores
        # each question have 4 options and each option has 50 context scores
        scores.append(list_of_scores)

    return scores

In [ ]:
train_challenge['score']= joining_scores(train_challenge, avd_score(model,tokenizer, train_challenge))
dev_challenge['score']= joining_scores(dev_challenge, avd_score(model,tokenizer, dev_challenge))
test_challenge['score']= joining_scores(test_challenge, avd_score(model,tokenizer, test_challenge))

100%|██████████| 1172/1172 [20:53<00:00,  1.07s/it]


In [ ]:
pickle.dump((train_challenge, dev_challenge, test_challenge),
            open('/content/drive/MyDrive/arc_dataset_challnege_final_scores.pkl','wb'))

In [ ]:
train_easy['score']= joining_scores(train_easy, avd_score(model,tokenizer, train_easy))
dev_easy['score']= joining_scores(dev_easy, avd_score(model, tokenizer,dev_easy))
test_easy['score']= joining_scores(test_easy, avd_score(model,tokenizer, test_easy))

100%|██████████| 2376/2376 [37:02<00:00,  1.07it/s]


In [ ]:
pickle.dump((train_easy, dev_easy, test_easy),
            open('/content/drive/MyDrive/arc_dataset_easy_final_scores.pkl','wb'))

In [ ]:
train_challenge, dev_challenge, test_challenge= \
pickle.load(open('/content/drive/MyDrive/arc_dataset_challnege_final_scores.pkl', 'rb'))

train_easy_old, dev_easy_old, test_easy_old= \
pickle.load(open('/content/drive/MyDrive/arc_dataset_easy_final_scores.pkl', 'rb'))